In [13]:
import pandas as pd
import numpy as np

from sportsreference.nfl.teams import Teams
from sportsreference.nfl.boxscore import Boxscore
from sportsreference.nfl.roster import Player



# Step 1. Get Data.


In [7]:
for team in Teams():
    print(team.name)
    schedule = team.schedule  # Request the current team's schedule
    for game in schedule:
        print(game.date, game.points_scored, game.points_allowed)


Baltimore Ravens
September 8 59 10
September 15 23 17
September 22 28 33
September 29 25 40
October 6 26 23
October 13 23 17
October 20 30 16
November 3 37 20
November 10 49 13
November 17 41 7
November 25 45 6
December 1 20 17
December 8 24 17
December 12 42 21
December 22 31 15
December 29 28 10
January 11 0 None
San Francisco 49ers
September 8 31 17
September 15 41 17
September 22 24 20
October 7 31 3
October 13 20 7
October 20 9 0
October 27 51 13
October 31 28 25
November 11 24 27
November 17 36 26
November 24 37 8
December 1 17 20
December 8 48 46
December 15 22 29
December 21 34 31
December 29 26 21
January 11 0 None
Tampa Bay Buccaneers
September 8 17 31
September 12 20 14
September 22 31 32
September 29 55 40
October 6 24 31
October 13 26 37
October 27 23 27
November 3 34 40
November 10 30 27
November 17 17 34
November 24 35 22
December 1 28 11
December 8 38 35
December 15 38 17
December 21 20 23
December 29 22 28
New Orleans Saints
September 9 30 28
September 15 9 27
Septembe

September 9 16 24
September 15 14 16
September 22 16 27
September 29 24 26
October 6 20 13
October 13 16 0
October 17 6 30
October 27 13 15
November 3 24 19
November 17 23 27
November 24 3 20
December 1 23 20
December 8 38 24
December 15 3 23
December 22 27 17
December 29 16 15
Chicago Bears
September 5 3 10
September 15 16 14
September 23 31 15
September 29 16 6
October 6 21 24
October 20 25 36
October 27 16 17
November 3 14 22
November 10 20 13
November 17 7 17
November 24 19 14
November 28 24 20
December 5 31 24
December 15 13 21
December 22 3 26
December 29 21 19
Cincinnati Bengals
September 8 20 21
September 15 17 41
September 22 17 21
September 30 3 27
October 6 23 26
October 13 17 23
October 20 17 27
October 27 10 24
November 10 13 49
November 17 10 17
November 24 10 16
December 1 22 6
December 8 19 27
December 15 13 34
December 22 35 38
December 29 33 23
New York Jets
September 8 16 17
September 16 3 23
September 22 14 30
October 6 6 31
October 13 24 22
October 21 0 33
October 

In [8]:
vars(game)

{'_year': 2019,
 '_week': '17',
 '_day': 'Sun',
 '_date': 'December 29',
 '_boxscore': '201912290dal',
 '_type': 'Reg',
 '_datetime': None,
 '_result': 'L',
 '_overtime': '',
 '_location': '@',
 '_opponent_abbr': 'DAL',
 '_opponent_name': 'Dallas Cowboys',
 '_points_scored': '16',
 '_points_allowed': '47',
 '_pass_completions': '18',
 '_pass_attempts': '37',
 '_pass_yards': '183',
 '_pass_touchdowns': '1',
 '_interceptions': '1',
 '_times_sacked': '2',
 '_yards_lost_from_sacks': '23',
 '_pass_yards_per_attempt': '5.6',
 '_pass_completion_rate': '48.6',
 '_quarterback_rating': '61.0',
 '_rush_attempts': '20',
 '_rush_yards': '88',
 '_rush_yards_per_attempt': '4.4',
 '_rush_touchdowns': '0',
 '_field_goals_made': '3',
 '_field_goals_attempted': '3',
 '_extra_points_made': '1',
 '_extra_points_attempted': '1',
 '_punts': '4',
 '_punt_yards': '227',
 '_third_down_conversions': '1',
 '_third_down_attempts': '12',
 '_fourth_down_conversions': '0',
 '_fourth_down_attempts': '3',
 '_time_of_po

In [12]:
bs = Boxscore('201912290dal')
vars(bs)

{'_uri': '201912290dal',
 '_date': 'Sunday Dec 29, 2019',
 '_time': '4:25pm',
 '_stadium': 'AT&T Stadium',
 '_attendance': '90646',
 '_duration': '3:14',
 '_away_name': [<a>],
 '_home_name': [<a>],
 '_winner': None,
 '_winning_name': None,
 '_winning_abbr': None,
 '_losing_name': None,
 '_losing_abbr': None,
 '_away_points': '16',
 '_away_first_downs': '13',
 '_away_rush_attempts': '20-88-0',
 '_away_rush_yards': '20-88-0',
 '_away_rush_touchdowns': '20-88-0',
 '_away_pass_completions': '18-37-206-1-1',
 '_away_pass_attempts': '18-37-206-1-1',
 '_away_pass_yards': '18-37-206-1-1',
 '_away_pass_touchdowns': '18-37-206-1-1',
 '_away_interceptions': '18-37-206-1-1',
 '_away_times_sacked': '2-23',
 '_away_yards_lost_from_sacks': '2-23',
 '_away_net_pass_yards': '183',
 '_away_total_yards': '271',
 '_away_fumbles': '1-1',
 '_away_fumbles_lost': '1-1',
 '_away_turnovers': '2',
 '_away_penalties': '2-10',
 '_away_yards_from_penalties': '2-10',
 '_away_third_down_conversions': '1-12',
 '_away_

In [14]:
lm = Player('JackLa00')


{'_most_recent_season': '2019',
 '_index': 2,
 '_player_id': 'JackLa00',
 '_season': ['2018', '2019', 'Career'],
 '_name': 'Lamar Jackson',
 '_team_abbreviation': ['BAL', 'BAL', ''],
 '_position': ['qb', 'QB', ''],
 '_height': '6-2',
 '_weight': '212lb',
 '_birth_date': '1997-01-07',
 '_games': ['16', '15', '31'],
 '_games_started': ['7', '15', '22'],
 '_approximate_value': ['8', '26', '34'],
 '_qb_record': ['6-1-0', '13-2-0', '19-3-0'],
 '_completed_passes': ['99', '265', '364'],
 '_attempted_passes': ['170', '401', '571'],
 '_passing_completion': ['58.2', '66.1', '63.7'],
 '_passing_yards': ['1201', '3127', '4328'],
 '_passing_touchdowns': ['6', '36', '42'],
 '_passing_touchdown_percentage': ['3.5', '9.0', '7.4'],
 '_interceptions_thrown': ['3', '6', '9'],
 '_interception_percentage': ['1.8', '1.5', '1.6'],
 '_longest_pass': ['74', '83', '83'],
 '_passing_yards_per_attempt': ['7.1', '7.8', '7.6'],
 '_adjusted_yards_per_attempt': ['7.0', '8.9', '8.3'],
 '_yards_per_completed_pass': ['